# Одобеску 6132 ЛР №2

In [1]:
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, DateType

In [2]:
from pyspark.sql import SparkSession
import os

# нужно для чтения xml
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'

spark_session = SparkSession.builder.appName("Lab2").getOrCreate()
sc = spark_session._sc
spark_session

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


Посмотрим, что находится в файле programming-languages.csv

In [3]:
!head /mnt/data/programming-languages.csv

name,wikipedia_url
A# .NET,https://en.wikipedia.org/wiki/A_Sharp_(.NET)
A# (Axiom),https://en.wikipedia.org/wiki/A_Sharp_(Axiom)
A-0 System,https://en.wikipedia.org/wiki/A-0_System
A+,https://en.wikipedia.org/wiki/A%2B_(programming_language)
A++,https://en.wikipedia.org/wiki/A%2B%2B
ABAP,https://en.wikipedia.org/wiki/ABAP
ABC,https://en.wikipedia.org/wiki/ABC_(programming_language)
ABC ALGOL,https://en.wikipedia.org/wiki/ABC_ALGOL
ABSET,https://en.wikipedia.org/wiki/ABSET


Прочитаем programming-languages.csv и загрузим его в датафрейм

In [4]:
struct_type = StructType([StructField("Language", StringType(), False),StructField("Url", StringType(), False)])

df = spark_session.read.csv("/mnt/data/programming-languages.csv", schema=struct_type)
df.head(2)

[Row(Language='name', Url='wikipedia_url'),
 Row(Language='A# .NET', Url='https://en.wikipedia.org/wiki/A_Sharp_(.NET)')]

In [5]:
lang_list = [str(row[0]) for row in df.select('Language').collect()][1:]
print(lang_list[:10])

['A# .NET', 'A# (Axiom)', 'A-0 System', 'A+', 'A++', 'ABAP', 'ABC', 'ABC ALGOL', 'ABSET', 'ABSYS']


Посмотрим, что из себя представляет posts_sample.xml

In [6]:
!head -n 3 /mnt/data/posts_sample.xml

���<?xml version="1.0" encoding="utf-8"?>
<posts>
  <row Id="4" PostTypeId="1" AcceptedAnswerId="7" CreationDate="2008-07-31T21:42:52.667" Score="630" ViewCount="42817" Body="&lt;p&gt;I want to use a track-bar to change a form's opacity.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;This is my code:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;decimal trans = trackBar1.Value / 5000;&#xA;this.Opacity = trans;&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#xA;&lt;p&gt;When I build the application, it gives the following error:&lt;/p&gt;&#xA;&#xA;&lt;blockquote&gt;&#xA;  &lt;p&gt;Cannot implicitly convert type &lt;code&gt;'decimal'&lt;/code&gt; to &lt;code&gt;'double'&lt;/code&gt;&lt;/p&gt;&#xA;&lt;/blockquote&gt;&#xA;&#xA;&lt;p&gt;I tried using &lt;code&gt;trans&lt;/code&gt; and &lt;code&gt;double&lt;/code&gt; but then the control doesn't work. This code worked fine in a past VB.NET project.&lt;/p&gt;&#xA;" OwnerUserId="8" LastEditorUserId="3641067" LastEditorDisplayName="Rich B" LastEditDate="2019-07-19T01:39:54.173" 

Прочитаем posts_sample.xml, поместим его в DataFrame и выведем что-нибудь из него

In [7]:
posts_df = spark_session.read.format("xml").options(rowTag="row").load('/mnt/data/posts_sample.xml')
print(posts_df.take(1))

[Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUs

Видим, что среди всего прочего в этом объекте есть поле _Tags, в котором может содержаться информация о языке, который относится к посту. Будем считать, что популярность языка напрямую зависит от числа постов, в которых присутствует его тег.

Напишем вспомогательные функции:   
* Первая определяет, есть ли тэг какого-либо языка из списка среди тэгов к посту, если есть вернёт информацию о нём в формате (row._Id, tag, row._CreationDate.year), иначе вернёт None.   
* Вторая проверяет, что пост находится между 2010 и 2020 годом

In [8]:
def get_correct_posts(row):
    tag = None
    for lang in lang_list:
        if '<' + lang.upper() + '>' in row._Tags.upper():
            tag = lang
            break
    if tag is None: 
        return None  # такие строки мы потом отфильтруем
    return (row._Id, tag, row._CreationDate.year)

def date_in_range(row):
    left = datetime(year=2010, month=1, day=1)
    right = datetime(year=2019, month=12, day=31)
    return row._CreationDate > left and row._CreationDate < right

Чтобы получить количество постов по каждому языку по годам, нужно выполнить ряд операций:
1. Проверяем, что тэг поста не None и что пост подходит нам по времени
2. С помощью get_correct_posts оставляем только информацию ((row._Id, tag, row._CreationDate.year)) о тех постах, в которых содержатся только тэги с языками
3. Удаляем те строки, где был возвращён None из предыдущей функции
4. В качестве ключа выбираем год создания поста и язык 
5. Считаем количество постов для пар (год, язык)
6. Ставим на первое место год, на второе - язык и на третье - количество постов
7. Преобразуем в датафрейм, указываем названия столбцов

In [9]:
langs_by_years = posts_df.rdd.filter(lambda row: row._Tags is not None and date_in_range(row))\
    .map(get_correct_posts)\
    .filter(lambda row: row is not None)\
    .keyBy(lambda row: (row[2], row[1]))\
    .aggregateByKey(0, lambda acc, value: acc + 1, lambda acc1, acc2: acc1 + acc2,)\
    .map(lambda row: (row[0][0], row[0][1], row[1]))\
    .toDF(('Year', 'Language', 'Posts'))
langs_by_years.show()  

+----+------------+-----+
|Year|    Language|Posts|
+----+------------+-----+
|2010|      Python|   25|
|2010|  JavaScript|   44|
|2010|           R|    3|
|2011| Objective-C|   33|
|2011|  JavaScript|   82|
|2013|  JavaScript|  196|
|2013|      Python|   87|
|2013|ActionScript|    1|
|2014|  JavaScript|  235|
|2014|      Python|  103|
|2014|         AWK|    5|
|2016|         PHP|  126|
|2016|  PowerShell|   11|
|2016|        Curl|    5|
|2016|       Scala|   16|
|2019|  JavaScript|  131|
|2019|          Go|    9|
|2019|        Curl|    3|
|2019|       Scala|    6|
|2015|      Python|  119|
+----+------------+-----+
only showing top 20 rows



Сохраним в папке Reports

In [10]:
path = "Reports/"

In [11]:
langs_by_years.write.mode("overwrite").parquet(path + "langs_by_years.parquet")

Сохраним теперь отдельно таблицы по годам, запросы выполним через SQL

In [12]:
folder_save_name = "Top_10_programming_languages_in_the_last_10_years"

parquet_df = spark_session.read.parquet(path + "langs_by_years.parquet")
parquet_df.createOrReplaceTempView("parquet_df")

for year in range(2010, 2020):
    print(year)
    top_10 = spark_session.sql(
        "SELECT Language, Posts "
        "FROM parquet_df "
        f"WHERE Year = {year} "
        "ORDER BY Posts DESC LIMIT 10 "
    )
    top_10.write.mode("overwrite").parquet(f"{path}{folder_save_name}/top_10_{year}.parquet")
    top_10.show()
    print('------------------')

2010
+-----------+-----+
|   Language|Posts|
+-----------+-----+
|       Java|   52|
| JavaScript|   44|
|        PHP|   42|
|     Python|   25|
|Objective-C|   23|
|          C|   20|
|       Ruby|   11|
|     Delphi|    7|
|       Bash|    3|
|AppleScript|    3|
+-----------+-----+

------------------
2011
+-----------+-----+
|   Language|Posts|
+-----------+-----+
|        PHP|   97|
|       Java|   92|
| JavaScript|   82|
|     Python|   35|
|Objective-C|   33|
|          C|   24|
|       Ruby|   17|
|     Delphi|    8|
|       Perl|    8|
|       Bash|    7|
+-----------+-----+

------------------
2012
+-----------+-----+
|   Language|Posts|
+-----------+-----+
|        PHP|  136|
| JavaScript|  129|
|       Java|  124|
|     Python|   65|
|Objective-C|   45|
|          C|   27|
|       Ruby|   25|
|       Bash|    9|
|          R|    9|
|        Lua|    6|
+-----------+-----+

------------------
2013
+-----------+-----+
|   Language|Posts|
+-----------+-----+
| JavaScript|  196|
